# Introduction to empirical market microstructure in Python

Prepared by [Vincent Grégoire](http://www.vincentgregoire.com), Department of Finance, The University of Melbourne. 

This is a sample code to illustrate how you can merge and analyze intraday financial data.

This code loads and analyzes intraday trades and quotes for all ASX20 stocks for the first two weeks of
March 2017 using
data from [Thomson Reuters Tick History](https://tickhistory.thomsonreuters.com/), provided by [SIRCA](https://www.sirca.org.au/). 

This notebook was created as supplemental material to a Python for financial research workshop for finance honours and Ph.D. students at the University of Melbourne in March of 2018.

Latest version: <https://github.com/vgreg/python-finance-unimelb2018>

**Outline**:

- Load trades and quotes
- Parsing timestamps (and convert from UTC)
- Cleaning, forward filling quotes
- Realized spread (merging asof)
- Time-weighted depth
- Plot something
- Summarize data at the stock/day level



In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
from datetime import datetime, date, time, timedelta
import matplotlib.pyplot as plt

The data requested from TRTH is in a file named "ASX20_HalfMarch2017.csv.gz". First, let's get a sense of the data.

In [2]:
pd.read_csv('ASX20_HalfMarch2017.csv.gz', nrows=100)

,#RIC,Current RIC,Date[L],Time[L],Type,Price,Volume,Bid Price,Bid Size,Ask Price,Ask Size,Qualifiers,Exch Time,Trd/Qte Date,Quote Time
0,AMP.AX,NaN,03/01/2017,06:10:21.249,Quote,NaN,NaN,4.86,1000.0,NaN,NaN,NaN,NaN,NaN,19:10:21.000
1,AMP.AX,NaN,03/01/2017,06:10:21.249,Quote,NaN,NaN,4.86,2000.0,NaN,NaN,NaN,NaN,NaN,19:10:21.000
2,AMP.AX,NaN,03/01/2017,06:10:21.249,Quote,NaN,NaN,NaN,NaN,4.89,2000.0,NaN,NaN,NaN,19:10:21.000
3,AMP.AX,NaN,03/01/2017,07:00:03.289,Quote,NaN,NaN,4.87,175.0,NaN,NaN,NaN,NaN,NaN,20:00:03.000
4,AMP.AX,NaN,03/01/2017,07:01:28.368,Quote,NaN,NaN,4.88,1000.0,NaN,NaN,NaN,NaN,NaN,20:01:28.000
5,AMP.AX,NaN,03/01/2017,07:03:18.538,Quote,NaN,NaN,NaN,NaN,4.91,1050.0,NaN,NaN,NaN,20:03:18.000
6,AMP.AX,NaN,03/01/2017,07:10:12.544,Quote,NaN,NaN,NaN,NaN,4.64,1759.0,NaN,NaN,NaN,20:10:12.000
7,AMP.AX,NaN,03/01/2017,08:30:05.654,Quote,NaN,NaN,5.27,195.0,NaN,NaN,NaN,NaN,NaN,21:30:05.000
8,AMP.AX,NaN,03/01/2017,08:30:10.604,Quote,NaN,NaN,5.27,173.0,NaN,NaN,NaN,NaN,NaN,21:30:10.000
9,AMP.AX,NaN,03/01/2017,08:48:48.448,Quote,NaN,NaN,5.27,225.0,NaN,NaN,NaN,NaN,NaN,21:48:48.000


In this case we won't use the TRTH timestamps (we'll use exchange timestamps). It will also be more practictal to split trades and quotes in different datasets.

In [3]:
cols = ['#RIC', 'Date[L]', 'Type', 'Price', 'Volume', 'Bid Price', 'Bid Size',
        'Ask Price', 'Ask Size', 'Qualifiers', 'Exch Time', 'Quote Time']
df = pd.read_csv('ASX20_HalfMarch2017.csv.gz', usecols=cols)

In [4]:
df_quotes = df.loc[df.Type=='Quote', ['#RIC', 'Date[L]','Bid Price', 'Bid Size',
                                      'Ask Price', 'Ask Size', 'Quote Time']].copy()
df_quotes.head()

,#RIC,Date[L],Bid Price,Bid Size,Ask Price,Ask Size,Quote Time
0,AMP.AX,03/01/2017,4.86,1000.0,NaN,NaN,19:10:21.000
1,AMP.AX,03/01/2017,4.86,2000.0,NaN,NaN,19:10:21.000
2,AMP.AX,03/01/2017,NaN,NaN,4.89,2000.0,19:10:21.000
3,AMP.AX,03/01/2017,4.87,175.0,NaN,NaN,20:00:03.000
4,AMP.AX,03/01/2017,4.88,1000.0,NaN,NaN,20:01:28.000


In [5]:
df_trades = df.loc[df.Type=='Trade', ['#RIC', 'Date[L]', 'Price', 'Volume', 'Qualifiers', 'Exch Time']].copy()
df_trades.head()

,#RIC,Date[L],Price,Volume,Qualifiers,Exch Time
63,AMP.AX,03/01/2017,4.86,1985.0,[ACT_FLAG1];MO [GV4_TEXT];Open|High|Low[USER],23:00:11.000
64,AMP.AX,03/01/2017,4.86,753.0,[ACT_FLAG1];MO [GV4_TEXT],23:00:11.000
65,AMP.AX,03/01/2017,4.86,424.0,[ACT_FLAG1];MO [GV4_TEXT],23:00:11.000
66,AMP.AX,03/01/2017,4.86,912.0,[ACT_FLAG1];MO [GV4_TEXT],23:00:11.000
67,AMP.AX,03/01/2017,4.86,618.0,[ACT_FLAG1];MO [GV4_TEXT],23:00:11.000


In [6]:
len(df_quotes)

5818262

In [7]:
len(df_trades)

2501299

To parse dates and timestamps, we'll use a simple trick that saves time by assuming a lot of dates and times are the same. We parse date and time separately since there is much more redundancy in dates.

In [8]:
def parse_simple_date(d):
    return datetime.strptime(d, '%m/%d/%Y')

def parse_simple_time(t):
    # The data is up to the second precision.
    return timedelta(hours=int(t[0:2]), minutes=int(t[3:5]),
                     seconds=int(t[6:8]))

def fast_date_parse(df, col, date_parser=parse_simple_date):
    dt = pd.DataFrame(df[col].unique())
    dt.columns = [col + '_tmp']
    dt[col] = dt[col + '_tmp'].apply(date_parser)
    date_dict = dt.set_index(col + '_tmp').to_dict()
    df[col] = df[col].map(date_dict[col])
    return df

In [ ]:
# To join dates and times, we need to account for time zone. Dates are local dates (Sydney),
# but times are in UTC.
df_trades = fast_date_parse(df_trades, 'Exch Time', parse_simple_time)
df_trades = fast_date_parse(df_trades, 'Date[L]')
df_trades['Timestamp'] = (df_trades['Date[L]'].dt.tz_localize('UTC').dt.tz_convert('Australia/Sydney') +
                          df_trades['Exch Time'])
sel = (df_trades.Timestamp.dt.date != df_trades['Date[L]'].dt.date)
df_trades.loc[sel, 'Timestamp'] = df_trades.loc[sel, 'Timestamp'] - timedelta(days=1)
df_trades.head()

,#RIC,Date[L],Price,Volume,Qualifiers,Exch Time,Timestamp
63,AMP.AX,2017-03-01,4.86,1985.0,[ACT_FLAG1];MO [GV4_TEXT];Open|High|Low[USER],23:00:11,2017-03-01 10:00:11+11:00
64,AMP.AX,2017-03-01,4.86,753.0,[ACT_FLAG1];MO [GV4_TEXT],23:00:11,2017-03-01 10:00:11+11:00
65,AMP.AX,2017-03-01,4.86,424.0,[ACT_FLAG1];MO [GV4_TEXT],23:00:11,2017-03-01 10:00:11+11:00
66,AMP.AX,2017-03-01,4.86,912.0,[ACT_FLAG1];MO [GV4_TEXT],23:00:11,2017-03-01 10:00:11+11:00
67,AMP.AX,2017-03-01,4.86,618.0,[ACT_FLAG1];MO [GV4_TEXT],23:00:11,2017-03-01 10:00:11+11:00


In [ ]:
df_quotes = fast_date_parse(df_quotes, 'Quote Time', parse_simple_time)
df_quotes = fast_date_parse(df_quotes, 'Date[L]')
df_quotes['Timestamp'] = (df_quotes['Date[L]'].dt.tz_localize('UTC').dt.tz_convert('Australia/Sydney') +
                          df_quotes['Quote Time'])
sel = (df_quotes.Timestamp.dt.date != df_quotes['Date[L]'].dt.date)
df_quotes.loc[sel, 'Timestamp'] = df_quotes.loc[sel, 'Timestamp'] - timedelta(days=1)
df_quotes.head()

As we can see, quotes get updated only one side at a time, so we need to fill forward the quotes.

We will first do the processing on one day only. It will be easier to see, and then we can write a function
with our code to loop over every day in the sample. This is usually a good approach; when the processing is split into logical elements (i.e. stock or day) it is easier to write and to run in parallel.

In [ ]:
quotes = df_quotes[df_quotes['Date[L]'] == datetime(2017, 3, 1)].copy()
trades = df_trades[df_trades['Date[L]'] == datetime(2017, 3, 1)].copy()

In [ ]:
quotes_cols = ['Bid Price', 'Bid Size', 'Ask Price', 'Ask Size']
quotes[quotes_cols] = quotes.groupby(['#RIC'])[quotes_cols].fillna(method='ffill')
# Missing values implie no depth
quotes[['Bid Size', 'Ask Size']] = quotes[['Bid Size', 'Ask Size']].fillna(0.0)
quotes.head()

Let's create a few measures.

In [ ]:
quotes['MidQuote'] = (quotes['Bid Price'] + quotes['Ask Price'])/2.0
quotes['DepthOnTop'] = (quotes['Bid Size'] + quotes['Ask Size'])/2.0
quotes['Spread'] = quotes['Ask Price'] - quotes['Bid Price']
quotes['RelSpread'] = quotes['Spread']/quotes['MidQuote']

In [ ]:
trades['Value'] = trades['Price'] * trades['Volume']

We want to focus on continuous trading.

We can look at trading hours on [ASX's website](http://www.asx.com.au/about/trading-hours.htm).

By 10:10 am, all stocks should be open, and they all stop at 4pm for an auction about 10 minutes later.

In [ ]:
# We will focus on continuous trading. 
start_time = time(hour=10, minute=10)
end_time = time(hour=16, minute=0)

In [ ]:
sel = ((trades.Timestamp.dt.time > start_time) &
       (trades.Timestamp.dt.time < end_time))

trades_open = trades[sel].copy()

In [ ]:
sel = ((trades.Timestamp.dt.time > start_time) &
       (trades.Timestamp.dt.time < end_time))

trades_open = trades[sel].copy()

In [ ]:
trades_open = trades_open.sort_values(['Timestamp', '#RIC'])
quotes = quotes.sort_values(['Timestamp', '#RIC'])

## Spreads

Two common measures for the cost of trading are the effective spread and the realized spread.

The definition for each of them is the following:

### Effective spread

Buy market order: $2\times (p_t - m_t)$

Sell market order: $2\times (m_t - p_t)$

where $p_t$ is the price for the transaction at time $t$, and $m_t$ is the mid quote at time $t-\varepsilon$ (just before the trade).


### Realized spread

Buy market order: $2\times (p_t - m_{t+\Delta})$

Sell market order: $2\times (m_{t+\Delta} - p_t)$

where $p_t$ is the price for the transaction at time $t$, and $m_{t+\Delta}$ is the mid quote at time ${t+\Delta}$. A common value for $\Delta$ is 5 minutes, so we'll use that.




First, we need to merge trades and quotes at the right time.

In [ ]:
trades_open['TS_5min'] = trades_open['Timestamp'] + timedelta(minutes=5)

In [ ]:
taq_merged = pd.merge_asof(trades_open,
                           quotes[['#RIC', 'Timestamp', 'Bid Price', 'Ask Price', 'MidQuote']],
                           on='Timestamp', by='#RIC', allow_exact_matches=False)

taq_merged = pd.merge_asof(taq_merged,
                           quotes[['#RIC', 'Timestamp', 'Bid Price', 'Ask Price', 'MidQuote']],
                           left_on='TS_5min', right_on='Timestamp',
                           by='#RIC', suffixes=('', '_5min'),
                           allow_exact_matches=False)

In [ ]:
taq_merged.head()

We next need to "sign" the trades, figure out if it's a buy or a sell.

In [ ]:
taq_merged['Sign'] = ''
taq_merged.loc[taq_merged.Price == taq_merged['Bid Price'], 'Sign'] = 'S'
taq_merged.loc[taq_merged.Price == taq_merged['Ask Price'], 'Sign'] = 'B'
taq_merged.loc[taq_merged.Price == taq_merged['MidQuote'], 'Sign'] = 'C' # Cross

In [ ]:
taq_merged.head()

### Effective Spread

Buy market order: $2\times (p_t - m_t)$

Sell market order: $2\times (m_t - p_t)$

where $p_t$ is the price for the transaction at time $t$, and $m_t$ is the mid quote at time $t-\varepsilon$ (just before the trade).

In [ ]:
# Create the variable for every obs
taq_merged['Effective Spread'] = np.nan
b_sel = taq_merged.Sign == 'B'
taq_merged.loc[b_sel, 'Effective Spread'] = 2*(taq_merged.loc[b_sel, 'Price'] -
                                               taq_merged.loc[b_sel, 'MidQuote'])
s_sel = taq_merged.Sign == 'S'
taq_merged.loc[s_sel, 'Effective Spread'] = 2*(taq_merged.loc[s_sel, 'MidQuote'] -
                                               taq_merged.loc[s_sel, 'Price'])

### Realized spread

Buy market order: $2\times (p_t - m_{t+\Delta})$

Sell market order: $2\times (m_{t+\Delta} - p_t)$

where $p_t$ is the price for the transaction at time $t$, and $m_{t+\Delta}$ is the mid quote at time ${t+\Delta}$. We use $\Delta=$5 minutes.

In [ ]:
# Create the variable for every obs
taq_merged['Realized Spread'] = np.nan
b_sel = taq_merged.Sign == 'B'
taq_merged.loc[b_sel, 'Realized Spread'] = 2*(taq_merged.loc[b_sel, 'Price'] -
                                              taq_merged.loc[b_sel, 'MidQuote_5min'])
s_sel = taq_merged.Sign == 'S'
taq_merged.loc[s_sel, 'Realized Spread'] = 2*(taq_merged.loc[s_sel, 'MidQuote_5min'] -
                                              taq_merged.loc[s_sel, 'Price'])

**Note**: here we might choose to do something special for trades in the last 5 minutes, such as recomputing realized spread using the closing auction price instead of the midquote. We won't do it, meaning we're using the prevailing mid-quote at market close.

### Price impact

The realized spread is a measure of price impact, i.e., it measures how trades move prices. If prices move in the direction of the trade, realized spread becomes smaller and possibly negative. We might expect that price impact is related to trade size. To make everything comparable, we need to make the measures "relative" (i.e., in %) to something, either price or mid-quote. We use the prevailing mid-quote at trade time.

In [ ]:
taq_merged['RelEffective Spread'] = taq_merged['Effective Spread'] / taq_merged['MidQuote']
taq_merged['RelRealized Spread'] = taq_merged['Realized Spread'] / taq_merged['MidQuote']

In [ ]:
taq_merged.plot.scatter(x='Value', y='RelRealized Spread');

Ok, so we have a few outliers, so it's hard to see what is going on. One way around this issue is to group trades in value deciles.

In [ ]:
taq_merged['Value_q10'] = pd.qcut(taq_merged['Value'], q=10, labels=False)

In [ ]:
val_q10_mean = taq_merged.groupby('Value_q10')[['RelEffective Spread', 'RelRealized Spread', 
                                                'Effective Spread', 'Realized Spread']].mean()

In [ ]:
val_q10_mean[['RelEffective Spread', 'RelRealized Spread']].plot()

In [ ]:
val_q10_mean[['Effective Spread', 'Realized Spread']].plot()

To get the average value for each stock/day, we can groupby.

In [ ]:
spd_mean = taq_merged.groupby(['#RIC', 'Date[L]'])[['RelEffective Spread', 'RelRealized Spread', 
                               'Effective Spread', 'Realized Spread']].mean()
spd_mean

In [ ]:
# To get trade value-weighted average instead, it's not much harder.
cols = ['RelEffective Spread', 'RelRealized Spread',
        'Effective Spread', 'Realized Spread']
for c in cols:
    taq_merged[c + '_VW'] = taq_merged[c] * taq_merged['Value']

for c in cols:
    spd_mean[c + '_VW'] = (taq_merged.groupby(['#RIC', 'Date[L]'])[c + '_VW'].sum() /
                          taq_merged.groupby(['#RIC', 'Date[L]'])['Value'].sum())
spd_mean

## Time-weighted measures

Another couple of interesting measures are the time-weighted spread and depth, which give an idea of the liquidity available. Since these are time-weighted, we take the average value, weighted by the duration of the spread or available depth.

In [ ]:
# First, we need to compute the time interval. We cannot use diff() since we want the difference
# between the next timestmap (when a new quote/depth is set) and the current timestamp.
quotes['time_d'] = (quotes.groupby('#RIC')['Quote Time'].shift(-1) - quotes['Quote Time']).dt.total_seconds()

In [ ]:
# Next, we do as before, using 'time_d' instead of Value for our weights.
cols = ['Spread', 'RelSpread', 'DepthOnTop']
for c in cols:
    quotes[c + '_TW'] = quotes[c] * quotes['time_d']

tw_mean = quotes.groupby(['#RIC', 'Date[L]'])[[c + '_TW' for c in cols]].sum()
    
for c in cols:
    tw_mean[c + '_TW'] = (tw_mean[c + '_TW'] /
                          quotes.groupby(['#RIC', 'Date[L]'])['time_d'].sum())
tw_mean

In [ ]:
# Now we can merge our two datasets

merged = pd.merge(spd_mean, tw_mean, left_index=True, right_index=True)
merged

# Exercise

Now that we have done all of this for one day, package as a function to loop over every day in the sample (without the plots) 
and then merge to get a complete stock/day panel, and export it to CSV.